# Running ASFMAUS

In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays
using Distributions

In [2]:
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");
include("/home/callum/ASF/Modules/ASF_analysis.jl");

In [3]:
input_path = "/home/callum/ASF/Inputs/"

"/home/callum/ASF/Inputs/"

In [6]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

In [7]:
nt = input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 11 #number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [8]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
                ASF_Model.reparam!(input_1)
                remake(prob,p=input_1.Parameters)
end 

prob_func (generic function with 1 method)

## Running the Model

### Frequency run

In [58]:
input_1 = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

In [1]:
rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

sol = solve(jump_prob, SimpleTauLeaping(),dt=1, DiscreteCallback= cb)

#ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
#output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 10,dt=1);

UndefVarError: UndefVarError: ASF_Model not defined

## Running Multiple Networks

In [13]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    input_folder = splitpath(path)[end]
    base_path = rsplit(path, input_folder)[1]

    #making save directory
    dir = "$(base_path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    #copying inputs
    cp("$(path)", "Inputs",force = true)
    
    for i = 1:n_networks
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model
        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        println("$(100*i/n_networks) %")

    end
    
end

sensitivity (generic function with 1 method)

In [14]:
sensitivity(20,20,100,input_path, "Dataout") # running 

TaskFailedException: TaskFailedException

    nested task error: InterruptException:
    Stacktrace:
      [1] Array
        @ ./boot.jl:448 [inlined]
      [2] Array
        @ ./boot.jl:457 [inlined]
      [3] Array
        @ ./boot.jl:465 [inlined]
      [4] similar
        @ ./abstractarray.jl:785 [inlined]
      [5] similar
        @ ./abstractarray.jl:784 [inlined]
      [6] similar
        @ ./broadcast.jl:197 [inlined]
      [7] similar
        @ ./broadcast.jl:196 [inlined]
      [8] copy
        @ ./broadcast.jl:908 [inlined]
      [9] materialize
        @ ./broadcast.jl:883 [inlined]
     [10] density_rate(out::Vector{Float64}, u::Vector{Int64}, p::Main.ASF_Inputs.Model_Parameters, t::Float64)
        @ Main.ASF_Model ~/ASF/Modules/ASF_model.jl:50
     [11] solve(jump_prob::JumpProblem{true, DiscreteProblem{Vector{Int64}, Tuple{Float64, Float64}, true, Main.ASF_Inputs.Model_Parameters, DiscreteFunction{true, SciMLBase.var"#158#159", Nothing, Nothing}, Base.Iterators.Pairs{Un

## Analysing Output

In [ ]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Baseline//");
connect_5 = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Connect_5//");


In [ ]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

In [ ]:
connect_5_r = ASF_Analysis.three_statistics(connect_5,20, p_out = true);